In [1]:
import pandas as pd

In [2]:
from preprocessing.logParser import openLogs
from content.video import Video

In [3]:
cursor = openLogs('logs/spbu_ACADRU_spring_2018-TL')

In [4]:
request = """
    select *
    from play_video
"""

cursor.execute(request)
data = cursor.fetchall()

columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])

df = pd.DataFrame(data=data, columns=columns_names)
df.head(5)

,user_id,course_name,execute_time,page,video_time
0,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:11:17.384343+00:00,https://courses.openedu.ru/courses/course-v1:s...,0.008867
1,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:11:40.543950+00:00,https://courses.openedu.ru/courses/course-v1:s...,22.540000
2,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:37.350351+00:00,https://courses.openedu.ru/courses/course-v1:s...,0.000000
3,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:46.876583+00:00,https://courses.openedu.ru/courses/course-v1:s...,8.960000
4,592408,course-v1:spbu+ACADRU+spring_2018,2018-03-01T06:18:49.363691+00:00,https://courses.openedu.ru/courses/course-v1:s...,36.000000


In [5]:
def get_videos_pages():
    """get the set of pages y load_video event"""
    
    request = """
            select *
            from load_video
            """
    cursor.execute(request)
    data = cursor.fetchall()
    
    columns_names = []
    for i in cursor.description[:]:
        columns_names.append(i[0])
    
    df = pd.DataFrame(data=data, columns=columns_names)
    
    pages = set()
    for page in df.page:
        pages.add(page)
    return pages

In [6]:
def check(page):
    video_events = ["hide_transcript", "pause_video",
                    "play_video", "seek_video",
                    "show_transcript", "speed_change_video", "stop_video",
                    "video_hide_cc_menu", "video_show_cc_menu"]

    for i in video_events:
        request = """
                    select *
                    from {}
                    where page = "{}"
                    """.format(i, page)
        cursor.execute(request)
        data = cursor.fetchall()
        if data : return True
    return False

In [7]:
for i in get_videos_pages():
    print(i)

https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/3a57e4c7f4b1485f940ed66ddcd02039/43402fe600d4477dafb3a82fc68e5303/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/e8ef3195d2494854b8c8eb0fb8a8dd53/3ec5eca88b874192b2dac9cc2e129451/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/3a57e4c7f4b1485f940ed66ddcd02039/036a89273d3b4dabba1d3888c73cdb8d/
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/7f4d3d8d4b1a4e4b853deb4325167221/8e7b046a5801460799d0e36ed61bdf71/
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/461e5748b4d54766ab9ac70dcc1b35bc/54a0ccd230a14063aaf9486735abb993/
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/e8ef3195d2494854b8c8eb0fb8a8dd53/318da92b67b34ac293dace9237a3bd33/
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443

In [8]:
videos = []
for page in get_videos_pages():
    video = Video(page)
    videos.append(video)
    
    # get length by stop_video event
    request = """
            select max(video_time)
            from stop_video
            where page = "{}"
            """.format(page)
    cursor.execute(request)
    length = cursor.fetchall()[0]
    length = length[0]
    if length == None:      print(page)

https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/efafd51836be456eab57bddb82ec213c/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/d331d71c92f34c28bb1793382368f8b8/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/40b895532e1c45c4b714155088bf12b9/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/e8ef3195d2494854b8c8eb0fb8a8dd53/cb54f86cc9a94320b6e7bd5d8e3e875f/?child=last
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/db2ef2a59bf443e3a0c24188e2442511/2d09ec9e5c26402e9214f6d1a8699aeb/?child=first
https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/7e6fb4e876e640ecb005e32e23e3292a/f8ec5a6f8407442ea637c8bc3b7e977a/?child=last
https://courses.openedu.ru/courses/course-v1:spb

In [9]:
s = "https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/64a281fe82e642dd9c9cef40b8e788ec/#6-child"
s.split("#")

['https://courses.openedu.ru/courses/course-v1:spbu+ACADRU+spring_2018/courseware/b434df438c2045eba63afc089d64b3de/64a281fe82e642dd9c9cef40b8e788ec/',
 '6-child']

In [10]:
request = """
        select *
        from seek_video
        where page = "{}"
        """.format(page)
cursor.execute(request)
data = cursor.fetchall()
columns_names = []
for i in cursor.description[:]:
    columns_names.append(i[0])
df = pd.DataFrame(data=data, columns=columns_names)
df[df.video_new_time < df.video_old_time]

,user_id,course_name,execute_time,page,video_old_time,video_new_time
0,560618,course-v1:spbu+ACADRU+spring_2018,2018-04-26T07:28:53.290647+00:00,https://courses.openedu.ru/courses/course-v1:s...,90.125354,37.0
1,543485,course-v1:spbu+ACADRU+spring_2018,2018-04-26T08:38:50.838181+00:00,https://courses.openedu.ru/courses/course-v1:s...,421.844464,285.0
2,543485,course-v1:spbu+ACADRU+spring_2018,2018-04-26T08:38:59.396106+00:00,https://courses.openedu.ru/courses/course-v1:s...,291.423928,243.0
3,265517,course-v1:spbu+ACADRU+spring_2018,2018-05-02T12:40:58.145256+00:00,https://courses.openedu.ru/courses/course-v1:s...,312.053542,215.0
4,265517,course-v1:spbu+ACADRU+spring_2018,2018-05-02T12:41:04.042710+00:00,https://courses.openedu.ru/courses/course-v1:s...,221.768250,197.0
5,265517,course-v1:spbu+ACADRU+spring_2018,2018-05-02T12:49:34.559025+00:00,https://courses.openedu.ru/courses/course-v1:s...,209.000000,160.0
10,596293,course-v1:spbu+ACADRU+spring_2018,2018-05-03T18:58:24.567359+00:00,https://courses.openedu.ru/courses/course-v1:s...,293.680873,280.0
11,596293,course-v1:spbu+ACADRU+spring_2018,2018-05-03T18:58:33.958245+00:00,https://courses.openedu.ru/courses/course-v1:s...,284.326358,279.0
12,596293,course-v1:spbu+ACADRU+spring_2018,2018-05-03T18:58:34.735469+00:00,https://courses.openedu.ru/courses/course-v1:s...,279.344287,270.0
13,596293,course-v1:spbu+ACADRU+spring_2018,2018-05-03T18:58:35.873437+00:00,https://courses.openedu.ru/courses/course-v1:s...,270.874104,260.0
